<a href="https://colab.research.google.com/github/muhammadali74/My_CUDA_Repo/blob/main/Week4/Simple2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc4jupyter

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-q04kn03k
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-q04kn03k
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 5cd225851b7638f3f6d55a19328295f16c014079
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.0.3-py3-none-any.whl size=7432 sha256=ad99cf1e6175f37f13398802e9e82d42885745e385f9d1c2ce678f3d359f3a95
  Stored in directory: /tmp/pip-ephem-wheel-cache-7zos_h3q/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter
Source files will be saved in "/tmp/tmprcpoicla".


In [12]:
%%cuda
//Taken from: https://code.google.com/p/stanford-cs193g-sp2010/wiki/TutorialMultidimensionalKernelLaunch

#include <stdlib.h>
#include <stdio.h>

__global__ void kernel(int *array)
{
  int index_x = blockIdx.x * blockDim.x + threadIdx.x;
  int index_y = blockIdx.y * blockDim.y + threadIdx.y;

  // map the two 2D indices to a single linear, 1D index
  int grid_width = gridDim.x * blockDim.x;
  int index = index_y * grid_width + index_x;

  // map the two 2D block indices to a single linear, 1D block index
  int result = blockIdx.y * gridDim.x + blockIdx.x;

  // write out the result
  // array[index] = result;
  // array[index] = result*(blockDim.x * blockDim.y) + threadIdx.y * blockDim.y + threadIdx.x;
  array[index] = blockDim.y* (result*blockDim.x + threadIdx.y) + threadIdx.x;
}

int main(void)
{
  int num_elements_x = 16;
  int num_elements_y = 16;

  int num_bytes = num_elements_x * num_elements_y * sizeof(int);

  int *device_array = 0;
  int *host_array = 0;

  // allocate memory in either space
  host_array = (int*)malloc(num_bytes);
  cudaMalloc((void**)&device_array, num_bytes);

  // create two dimensional 4x4 thread blocks
  dim3 block_size;
  block_size.x = 4;
  block_size.y = 4;

  // configure a two dimensional grid as well
  dim3 grid_size;
  grid_size.x = num_elements_x / block_size.x;
  grid_size.y = num_elements_y / block_size.y;

  // grid_size & block_size are passed as arguments to the triple chevrons as usual
  kernel<<<grid_size,block_size>>>(device_array);

  // download and inspect the result on the host:
  cudaMemcpy(host_array, device_array, num_bytes, cudaMemcpyDeviceToHost);

  // print out the result element by element
  for(int row = 0; row < num_elements_y; ++row)
  {
    for(int col = 0; col < num_elements_x; ++col)
    {
      printf("%2d ", host_array[row * num_elements_x + col]);
    }
    printf("\n");
  }
  printf("\n");

  // deallocate memory
  free(host_array);
  cudaFree(device_array);
}

 0  1  2  3 16 17 18 19 32 33 34 35 48 49 50 51 
 4  5  6  7 20 21 22 23 36 37 38 39 52 53 54 55 
 8  9 10 11 24 25 26 27 40 41 42 43 56 57 58 59 
12 13 14 15 28 29 30 31 44 45 46 47 60 61 62 63 
64 65 66 67 80 81 82 83 96 97 98 99 112 113 114 115 
68 69 70 71 84 85 86 87 100 101 102 103 116 117 118 119 
72 73 74 75 88 89 90 91 104 105 106 107 120 121 122 123 
76 77 78 79 92 93 94 95 108 109 110 111 124 125 126 127 
128 129 130 131 144 145 146 147 160 161 162 163 176 177 178 179 
132 133 134 135 148 149 150 151 164 165 166 167 180 181 182 183 
136 137 138 139 152 153 154 155 168 169 170 171 184 185 186 187 
140 141 142 143 156 157 158 159 172 173 174 175 188 189 190 191 
192 193 194 195 208 209 210 211 224 225 226 227 240 241 242 243 
196 197 198 199 212 213 214 215 228 229 230 231 244 245 246 247 
200 201 202 203 216 217 218 219 232 233 234 235 248 249 250 251 
204 205 206 207 220 221 222 223 236 237 238 239 252 253 254 255 


